## PromptEngineering

In [2]:
from langchain import PromptTemplate

# The process of prompt Engineering is 
# Persona
# Task
# Constraints
# Conditions
# Goal
# Format
# Reasoning

demo_template = '''
I want you to act as a financial advisor for a client who is looking to invest in the stock market.
In an easy way, explain he basics of {financial_concept} to a beginner.
'''

In [4]:
initial_prompt = PromptTemplate(input_variable = ['financial_concept'],
                                template = demo_template)

In [5]:
initial_prompt.format(financial_concept = 'stock market')

'\nI want you to act as a financial advisor for a client who is looking to invest in the stock market.\nIn an easy way, explain he basics of stock market to a beginner.\n'

In [7]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain

In [8]:
llm = OllamaLLM(
    model = "llama3.2:latest",
    temperature = 0.8
)

In [10]:
chain_1 = LLMChain(llm = llm, prompt = initial_prompt)

In [12]:
chain_1.run("stock market")

"Congratulations on taking the first step towards investing in the stock market! As your financial advisor, I'm here to guide you through the basics.\n\n**What is the Stock Market?**\n\nThe stock market, also known as the equity market, is a platform where companies raise capital by issuing shares of stock to the public. These shares represent ownership in the company and give investors a claim on its assets and profits.\n\n**How Does it Work?**\n\nImagine you own a small business, and you want to grow your company but need more money. You can sell shares of your company to others, who become part-owners of your business. In return, they pay you a portion of the profits. This way, the owners (shareholders) receive a share of the company's success.\n\nThe stock market works similarly:\n\n1. Companies issue stocks to raise capital.\n2. Investors buy these shares and become part-owners of the company.\n3. The value of the shares can fluctuate based on various factors like the company's pe

In [25]:
# Language Translation

lang_template = '''
In an easy way translate the following text from {sentence} into {target_language}. Just give the translation in a respectful and professional manner.
'''

lang_prompt = PromptTemplate(input_variable = ['sentence', 'target_language'],
                             template = lang_template)

lang_prompt.format(sentence = 'How are you?', target_language = 'French')

'\nIn an easy way translate the following text from How are you? into French. Just give the translation in a respectful and professional manner.\n'

In [26]:
chain_2 = LLMChain(llm = llm, prompt = lang_prompt)

In [27]:
chain_2({ "sentence": "How are you?", "target_language" : "Spanish"})

{'sentence': 'How are you?',
 'target_language': 'Spanish',
 'text': '"How are you?" translates to "¿Cómo estás?"\n\nPlease note that this is a common greeting used in many countries, but it\'s always best to learn regional variations and expressions for more effective communication.\n\nAlternatively, a more formal way of saying "How are you?" in Spanish would be:\n\n"¿Está bien?"\n\nOr, if you want to be more friendly and informal:\n\n"Hola, ¿cómo estás hoy?"\n\nThis second option is commonly used among friends or acquaintances.'}

In [28]:
from langchain import FewShotPromptTemplate

In [37]:
examples = [
    {"word" : "happy", "antonym" : "sad"},
    {"word" : "hot", "antonym" : "cold"},
    {"word" : "good", "antonym" : "bad"},
]

example_format_template = '''word : {word}
antonym : {antonym}
'''

example_prompt = PromptTemplate(
    input_variable = ['word', 'antonym'],
    template = example_format_template
)

In [41]:
few_shot_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input\n",
    suffix = "Word: {input}\nAntonym:",
    input_variables = ['input'],
    example_separator = '\n',
)


In [42]:
print(few_shot_template.format(input = 'big'))

Give the antonym of every input

word : happy
antonym : sad

word : hot
antonym : cold

word : good
antonym : bad

Word: big
Antonym:


In [43]:
chain_3 = LLMChain(llm = llm, prompt = few_shot_template)

chain_3('big')

{'input': 'big', 'text': 'The antonym for "big" is "small".'}

## PDF Query using Lancgchain

In [45]:
%%capture
!pip install langchain
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [46]:
from PyPDF2 import PdfReader
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [47]:
pdfreader = PdfReader("../files/budget_speech.pdf")

In [48]:
from typing_extensions import Concatenate

raw_txt = ""

for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_txt += page.extract_text()

In [49]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)

texts = text_splitter.split_text(raw_txt)

In [50]:
len(texts)

160

In [51]:
# embeddings
embeddings = OllamaEmbeddings(model = "llama3.2:latest")

In [53]:
doc_search = FAISS.from_texts(texts = texts, embedding = embeddings)

In [54]:
from langchain.chains.question_answering import load_qa_chain

In [89]:
chain = load_qa_chain(OllamaLLM(model = "llama3.2:latest"), chain_type="stuff")

In [94]:
query = "Which financial year's budget speech is this?"
docs = doc_search.similarity_search(query = query)

In [95]:
print(chain.run(input_documents = docs, question = query))

Based on the provided context, it appears that the budget speech was for the 2024-25 fiscal year, as mentioned in B.6, where "w.e.f. 1st August, 2024" refers to the time limit modification for filing appeals before the Appellate Tribunal.
